In [1]:
# -*- coding: utf-8 -*-

"""
This MILP (Mixed-integer linear programming) finds the solution that minimizes the cost of a telecom network given specific constraints
Run the main() function to print the solution
"""

def main():

    import pulp
    import pandas as pd
    import numpy as np

    # Input data preperation
    # The excel file must be in the same folder, othwise the path to the excel file must be specified
    filename = "InputDataTelecomSmallInstance.xlsx"
    C = pd.read_excel(filename, sheet_name="C", header=None)
    M = pd.read_excel(filename, sheet_name="M", header=None)
    N = pd.read_excel(filename, sheet_name="N", header=None)
    h = pd.read_excel(filename, sheet_name="CustToTargetAllocCost(hij)", header=None)
    c = pd.read_excel(filename, sheet_name="TargetToSteinerAllocCost(cjk)", header=None)
    g = pd.read_excel(filename, sheet_name="SteinerToSteinerConnctCost(gkm)", header=None)
    f = pd.read_excel(filename, sheet_name="SteinerFixedCost(fk)", header=None)
    U = pd.read_excel(filename, sheet_name="TargetCapicity(Uj)", header=None)
    V = pd.read_excel(filename, sheet_name="SteinerCapacity(Vk)", header=None)
    alpha = pd.read_excel(filename, sheet_name="alpha", header=None)

    C = C[0][0]
    M = M[0][0]
    N = N[0][0]
    alpha = alpha[0][0]
    h = np.array(h)
    c = np.array(c)
    g = np.array(g)
    f = np.array(f)
    U = np.array(U)
    V = np.array(V)

    f = f.flatten()
    U = U.flatten()
    V = V.flatten()

    set_C = np.arange(C) # set of customers
    set_M = np.arange(M) # set of end offices
    set_N = np.arange(N) # set of digital hubs
    # The values of the sets start at 0 (and not 1)

    # Decision variables
    X_var = pulp.LpVariable.dicts('X', (set_C, set_M), cat='Binary') # links between customers and end offices
    Y_var = pulp.LpVariable.dicts('Y', (set_M, set_N), cat='Binary') # links between end offices and digital hubs
    Z_var = pulp.LpVariable.dicts('Z', (set_N, set_N), cat='Binary') # links between digital hubs
    L_var = pulp.LpVariable.dicts('L', set_N, cat='Binary') # selected digital hubs
    W_var = pulp.LpVariable.dicts('W', (set_C, set_M, set_N), cat='Binary') # W = X * Y

    # Variable to contain the problem data
    cost = pulp.LpProblem("cost", pulp.LpMinimize)

    # Objective function
    h_alloc_cost = pulp.lpSum(X_var[i][j] * h[i][j] for i in set_C for j in set_M)
    c_alloc_cost = pulp.lpSum(Y_var[j][k] * c[j][k] for j in set_M for k in set_N)
    allocation_cost = h_alloc_cost + c_alloc_cost
    location_cost = pulp.lpSum(L_var[k] * f[k] for k in set_N)
    connection_cost = pulp.lpSum(Z_var[k][m] * g[k][m] for k in set_N for m in set_N)

    cost += allocation_cost + location_cost + connection_cost

    # Constraint 1: Single allocation for customers
    for i in set_C:
        cost += pulp.lpSum(X_var[i][j] for j in set_M) <= 1

    # Constraint 2: Single allocation for end offices
    for j in set_M:
        cost += pulp.lpSum(Y_var[j][k] for k in set_N) == 1
        
    # Constraint 3: Allocation of end offices to located digital hubs
    for j in set_M:
        for k in set_N:
            cost += Y_var[j][k] <= L_var[k]

    # Constraint 4: Ring (tour) structure over located digital hubs
    for m in set_N:
        cost +=  pulp.lpSum(Z_var[k][m] + Z_var[m][k] for k in set_N) == 2 * L_var[m]

    # Constraint 5: Subtour elimination
    subsets_N = pulp.allcombinations(set_N, N - 1)
    for H in subsets_N:
        if len(H) >= 3:
            not_H = np.delete(set_N, H)
            sum_Z = pulp.lpSum(Z_var[k][m] for k in H for m in H)
            sum_L = pulp.lpSum(L_var[j] for j in H)
            for t in not_H:
                for l in H:
                    cost += sum_Z <= sum_L - L_var[t] - L_var[l] + 1

    # Constraint 6: End office capacity constraint
    for j in set_M:
        cost += pulp.lpSum(X_var[i][j] for i in set_C) <= U[j]

    # Constraint 7: Digital hub capacity constraint
    for i in set_C:
        for j in set_M:
            for k in set_N:
                cost += W_var[i][j][k] <= X_var[i][j]
                cost += W_var[i][j][k] <= Y_var[j][k]
                cost += W_var[i][j][k] >= X_var[i][j] + Y_var[j][k] - 1

    for k in set_N:
        cost += pulp.lpSum(W_var[i][j][k] for i in set_C for j in set_M) <= V[k]

    # Constraint 8: A ring must have at least three digital hubs
    cost += pulp.lpSum(L_var[k] for k in set_N) >= 3

    # Constraint 9: Covering constraints
    cost += pulp.lpSum(X_var[i][j] for i in set_C for j in set_M) >= alpha * C

    # Solution
    cost.solve()
    print("Status:", pulp.LpStatus[cost.status])
    print ("Objective cost value = ", pulp.value(cost.objective))
    for v in cost.variables():
        if v.varValue > 0:
            print(v.name, "=", v.varValue)

main()

Status: Optimal
Objective cost value =  5119.0
L_0 = 1.0
L_2 = 1.0
L_3 = 1.0
W_0_4_3 = 1.0
W_10_1_2 = 1.0
W_11_1_2 = 1.0
W_12_4_3 = 1.0
W_14_0_0 = 1.0
W_1_7_0 = 1.0
W_2_3_2 = 1.0
W_3_1_2 = 1.0
W_4_7_0 = 1.0
W_5_4_3 = 1.0
W_6_3_2 = 1.0
W_7_3_2 = 1.0
W_8_0_0 = 1.0
W_9_2_2 = 1.0
X_0_4 = 1.0
X_10_1 = 1.0
X_11_1 = 1.0
X_12_4 = 1.0
X_14_0 = 1.0
X_1_7 = 1.0
X_2_3 = 1.0
X_3_1 = 1.0
X_4_7 = 1.0
X_5_4 = 1.0
X_6_3 = 1.0
X_7_3 = 1.0
X_8_0 = 1.0
X_9_2 = 1.0
Y_0_0 = 1.0
Y_1_2 = 1.0
Y_2_2 = 1.0
Y_3_2 = 1.0
Y_4_3 = 1.0
Y_5_3 = 1.0
Y_6_2 = 1.0
Y_7_0 = 1.0
Z_0_3 = 1.0
Z_2_0 = 1.0
Z_3_2 = 1.0
